In [1]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

pd.set_option('display.max_rows', None)

In [2]:
# df = pd.read_csv('~/sorted_data.txt', sep='\t')

In [ ]:
model_file = 'saved/SASRec-Nov-22-2021_08-08-52.pth'
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_file)

29 Nov 18:03    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/pretrain
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 2048
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = exposure_time
s

In [4]:
if test_data.is_sequential:
    uid_list = test_data.dataset.inter_feat['user_id']
    positive_item = test_data.dataset.inter_feat['item_id']
else:
    uid_list = test_data.uid_list
    positive_item = test_data.uid2positive_item[uid_list]
    positive_item = torch.cat(list(positive_item))
assert(len(positive_item) == len(uid_list))
print(len(uid_list))
topk_score, topk_iid_list = full_sort_topk(uid_list, model, test_data, k=10, device=config['device'])
topk_score = topk_score.cpu()
topk_iid_list = topk_iid_list.cpu()
correct_mask = (positive_item.view(-1, 1) == topk_iid_list).any(1)

7946


/home/sankuai/recbole/recbole/utils/case_study.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  uid_series = torch.tensor(uid_series)


In [5]:
correct_user = uid_list[correct_mask]
correct_user

tensor([   1,    7,   14,   15,   24,   46,   56,   80,  117,  119,   88,  153,
         162,  169,  156,  190,  120,  213,  168,  224,  239,  241,  226,  249,
         257,  256,  268,  269,  320,  328,  341,  348,  356,  378,  382,  391,
         398,   58,  408,  413,  326,  431,  140,  460,  502,  512,  523,  427,
         575,  581,  338,  129,  465,  595,  297,  619,  608,  669,  687,  693,
         715,  719,  734,  737,  743,  754,  759,  800,  173,  182,  835,  852,
         864,  886,  179,  178,  897,  923,  827,  112,  186,  745,  633, 1033,
         840,  546, 1053, 1056, 1100, 1101,  650,  675, 1018, 1158, 1199, 1150,
        1106,  775, 1276, 1280,  361, 1300,  298, 1329, 1354, 1012, 1368, 1373,
        1386, 1392, 1426, 1450,  636,  657,   96,  525, 1370, 1468, 1504, 1519,
        1500, 1540,  688,  552, 1583, 1601, 1021,  713, 1622,  655, 1630, 1201,
        1254, 1680,  703, 1282, 1721, 1729, 1730,   83, 1746,  905,  824, 1268,
         565, 1815, 1434,   69,  970, 14

In [6]:
dataset.id2token('user_id', correct_user[0])

'885577978'

In [7]:
test_inter = test_data.dataset.inter_feat
inter = test_inter[test_inter['user_id'] == correct_user[0]]

In [8]:
print(inter['user_id'], inter['item_id'])

tensor([1]) tensor([67980])


In [9]:
dataset.id2token('item_id', inter['item_id'])

'1531523834_POI_HOTEL2'

In [10]:
df[df['user_id'] == 885577978]

,user_id,item_id,name,time,inter_type,user_loc
171570,885577978,160522272_POI_LVYOU,清河半岛温泉度假酒店,2021-05-29 00:04:42,click,117.0814 36.6822
171571,885577978,160522272_POI_LVYOU,清河半岛温泉度假酒店,2021-05-29 00:07:01,click,117.0814 36.6822
171572,885577978,193690229_POI_HOTEL2,麗泽城市酒店（奥体中心地铁站店）,2021-06-04 16:04:07,click,117.0309 36.6661
171573,885577978,192154048_POI_HOTEL2,巴黎世家精品主题公寓（北站华府店）,2021-06-08 17:48:41,click,117.0509 36.6662
171574,885577978,647689510_PRODUCT,私房写真私房照,2021-06-11 15:43:59,click,117.0309 36.6661
171575,885577978,180669183_POI_WAIMAI,禾言己蟹黄包生煎包（山师店）,2021-06-11 15:44:46,click,117.0309 36.6661
171576,885577978,185151730_POI_BNB,2号小双人间外窗空调挤挤更甜蜜山大洪家楼济南智慧大厦山大中心校区,2021-06-11 15:47:13,click,117.0309 36.6662
171577,885577978,14358874_PRODUCT,超跑：兰博基尼，法拉利等,2021-06-11 15:47:13,click,117.0309 36.6662
171578,885577978,180669183_POI_WAIMAI,禾言己蟹黄包生煎包（山师店）,2021-06-11 16:50:45,click,117.0318 36.6665
171579,885577978,1101528688_POI_WAIMAI,刘忙把子肉（泺源大街店）,2021-06-11 16:52:09,click,117.0309 36.6661
